# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd 
import requests 
import xlsxwriter 
import math 

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 126559453, 'calculationPrice': 'close', 'change': 0.67, 'changePercent': 0.00368, 'close': 177.09, 'closeSource': 'olfiicaf', 'closeTime': 1651923582736, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 178.91, 'delayedPriceTime': 1716719430499, 'extendedChange': 0.04, 'extendedChangePercent': 0.00023, 'extendedPrice': 177.98, 'extendedPriceTime': 1721137581305, 'high': 182.1, 'highSource': 'eClos', 'highTime': 1649293720571, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 184.8, 'iexCloseTime': 1666045752377, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 179.29, 'iexOpenTime': 1685182273847, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1720738103651, 'latestPrice': 182.02, 'latestSource': 'Close', 'latestTime': 'December 23, 2021', 'latestUpdate': 1651243256453, 'latestVolume': 68879257, 'low': 176.93, 'lowSource': 'Close', 'lowTime': 1649390199212, '

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = (data["latestPrice"])
market_cap = data["marketCap"]
print(price)
print(market_cap/1000000000000)

182.02
2.983384412032


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ("Ticker", "Stock Price", "Market Capitalisation", "Number of Shares to Buy")
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        "N/A"
    ],
    index = my_columns), #tells the append method which columns to add the data to 
    ignore_index=True #necessary whenever you're appending data to the dataframe 
)

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,AAPL,182.02,2983384412032,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks["Ticker"][:5]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series([symbol,
            data['latestPrice'],
            data['marketCap'],
            "N/A"],
        index = my_columns),
    ignore_index = True
    )

# for symbol in stocks["Ticker"][:5]:
#     print(symbol)


In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,162.70,49919477506,N/A
1,AAL,18.39,12410366664,N/A
2,AAP,236.64,14741159687,N/A
3,AAPL,177.82,2924896300554,N/A
4,ABBV,137.91,243715336927,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [21]:
symbol_groups = chunks(stocks["Ticker"], 100) 
#if we do stocks it would pass a dataframe but if we did stocks["Ticker"] it will pass a series
# print(list(symbol_groups))
symbol_groups = list(symbol_groups) #this converts the generator output to a list

#list of strings where comma seperated string of all the stocks in symbol_groups object
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
#     print(symbol_strings[i]) 

#resets the final_dataframe
final_dataframe = pd.DataFrame(columns = my_columns)

#loop through every string with our symbol_strings object and then use that string to execute a batch API request
for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
#next we parse the batch api call for data for every specific stock and then append that data to the pandas dataframe 
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                "N/A"
            ],
            index = my_columns),
            ignore_index=True
        )
    
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,163.70,49478600892,N/A
1,AAL,18.75,12072803579,N/A
2,AAP,232.90,14961215728,N/A
3,AAPL,178.76,2942606746300,N/A
4,ABBV,135.41,240581587241,N/A
...,...,...,...,...
500,YUM,137.33,40269213574,N/A
501,ZBH,133.14,27409248083,N/A
502,ZBRA,601.15,32622446438,N/A
503,ZION,63.69,10224511904,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [45]:
portfolio_size = input("What is your portfolio size: ")

try: 
    val = float(portfolio_size)
    print(val)

except ValueError:
    print("That's not a number! \nPlease try again")
    portfolio_size = input("What is your portfolio size: ")
    val = float(portfolio_size)
    

What is your portfolio size:  10000000


10000000.0


In [47]:
position_size = val/len(final_dataframe.index)
for i in range (0, len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])
final_dataframe
                

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,163.70,49478600892,120
1,AAL,18.75,12072803579,1056
2,AAP,232.90,14961215728,85
3,AAPL,178.76,2942606746300,110
4,ABBV,135.41,240581587241,146
...,...,...,...,...
500,YUM,137.33,40269213574,144
501,ZBH,133.14,27409248083,148
502,ZBRA,601.15,32622446438,32
503,ZION,63.69,10224511904,310


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [69]:
#xlsx writer has issues and created corrupted files so i used pandas instead

final_dataframe.to_excel("recommended trades.xlsx")

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

<bound method XlsxWriter.save of <pandas.io.excel._xlsxwriter.XlsxWriter object at 0x7ff024a19ca0>>

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: